# Laboratorio 3

#### Mario de León
#### Javier Valle

#### Import necesarios

In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Usando Feed Forward NN (Red simple)

In [7]:
# Cargar el dataset
data = pd.read_csv('data.csv') 


# Separando un poco el formato.
data['Year'] = data['Month'].apply(lambda x: int(x.split('-')[0]))
data['Month'] = data['Month'].apply(lambda x: int(x.split('-')[1]))

# Dividir los datos en características (X) y etiquetas (y)
X = data[['Year', 'Month']]
y = data['Sunspots']

# Conjunto de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalando los datos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creando la estructura.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),  # 2 características de entrada (Year, Month)
    tf.keras.layers.Dense(256, activation='relu'),  
    tf.keras.layers.Dense(128, activation='relu'),  
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(32, activation='relu'),  
    tf.keras.layers.Dense(16, activation='relu'),  
    tf.keras.layers.Dense(8, activation='relu'),  
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilando el modelo.
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenando el modelo
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluando el modelo.
loss = model.evaluate(X_test_scaled, y_test)
print("Pérdida en el conjunto de prueba:", loss)

# Realiznado predicciones.
predictions = model.predict(X_test_scaled)

Epoch 1/100
71/71 [==============================] - 3s 7ms/step - loss: 3461.0564 - val_loss: 1815.8304
Epoch 2/100
71/71 [==============================] - 0s 5ms/step - loss: 1817.1284 - val_loss: 1789.2990
Epoch 3/100
71/71 [==============================] - 0s 4ms/step - loss: 1797.7974 - val_loss: 1804.4264
Epoch 4/100
71/71 [==============================] - 0s 4ms/step - loss: 1777.0072 - val_loss: 1760.5714
Epoch 5/100
71/71 [==============================] - 0s 5ms/step - loss: 1781.0054 - val_loss: 1753.6160
Epoch 6/100
71/71 [==============================] - 0s 4ms/step - loss: 1771.8381 - val_loss: 1747.4498
Epoch 7/100
71/71 [==============================] - 0s 4ms/step - loss: 1770.1365 - val_loss: 1741.5610
Epoch 8/100
71/71 [==============================] - 0s 4ms/step - loss: 1760.6772 - val_loss: 1747.8131
Epoch 9/100
71/71 [==============================] - 0s 4ms/step - loss: 1771.9324 - val_loss: 1747.5498
Epoch 10/100
71/71 [==============================] - 0

##### Predicciones del modelo.

In [8]:
print("Predicciones: ", predictions)

Predicciones:  [[51.22948 ]
 [54.958775]
 [35.846706]
 [53.01295 ]
 [80.87667 ]
 [54.137352]
 [36.959652]
 [87.10136 ]
 [51.567055]
 [35.230278]
 [52.8727  ]
 [39.512974]
 [45.634   ]
 [48.617676]
 [35.966686]
 [56.462337]
 [79.93168 ]
 [61.162537]
 [51.76311 ]
 [49.272427]
 [75.215416]
 [41.93086 ]
 [53.715168]
 [37.428886]
 [32.618202]
 [54.912388]
 [31.867136]
 [57.986576]
 [56.14261 ]
 [35.847103]
 [55.42913 ]
 [38.83011 ]
 [48.803654]
 [33.631924]
 [40.070415]
 [32.788788]
 [38.71044 ]
 [54.40465 ]
 [53.107143]
 [42.608746]
 [58.30898 ]
 [52.174854]
 [72.70336 ]
 [54.975487]
 [47.453163]
 [58.01161 ]
 [52.20134 ]
 [27.422153]
 [33.265205]
 [57.316093]
 [44.622025]
 [45.392117]
 [57.98595 ]
 [56.728367]
 [78.35455 ]
 [30.045702]
 [33.87652 ]
 [38.342197]
 [86.31332 ]
 [60.35066 ]
 [55.83288 ]
 [55.175003]
 [57.84438 ]
 [47.680363]
 [82.39877 ]
 [50.311684]
 [84.74079 ]
 [66.0447  ]
 [29.673433]
 [48.14721 ]
 [43.923264]
 [38.480564]
 [47.223568]
 [42.160755]
 [45.748486]
 [34.65028

## Usando RNN

In [13]:
# Cargar el conjunto de datos
data = pd.read_csv('data.csv')  

# Cambio de formato
data['Year'] = data['Month'].apply(lambda x: int(x.split('-')[0]))
data['Month'] = data['Month'].apply(lambda x: int(x.split('-')[1]))

# Preparar los datos
# Usando la secuencia como la cantidad de meses.
sequence_length = 12
input_data = []
target_data = []
for i in range(len(data) - sequence_length):
    input_sequence = data[['Year', 'Month', 'Sunspots']].iloc[i : i + sequence_length].values
    target_value = data['Sunspots'].iloc[i + sequence_length]
    input_data.append(input_sequence)
    target_data.append(target_value)

input_data = np.array(input_data)
target_data = np.array(target_data)

# Dividiendo el conjunto de datos en entrenamiento y prueba.
train_size = int(0.8 * len(input_data))
X_train, X_test = input_data[:train_size], input_data[train_size:]
y_train, y_test = target_data[:train_size], target_data[train_size:]

# Creando la estructura de la RNN
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 3)),  # Entrada con 3 características (Year, Month, Sunspots)
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='relu', return_sequences=True),
    tf.keras.layers.Flatten(),  # Aplananando las secuencias para la capa densa
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilando el modelo.
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenando el modelo.
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluando el modelo.
loss = model.evaluate(X_test, y_test)
print("Pérdida en el conjunto de prueba:", loss)

Epoch 1/50
71/71 [==============================] - 6s 23ms/step - loss: 4855.1084 - val_loss: 899.0070
Epoch 2/50
71/71 [==============================] - 1s 11ms/step - loss: 413.1766 - val_loss: 632.1909
Epoch 3/50
71/71 [==============================] - 1s 11ms/step - loss: 447.1842 - val_loss: 586.9870
Epoch 4/50
71/71 [==============================] - 1s 11ms/step - loss: 377.7224 - val_loss: 603.0665
Epoch 5/50
71/71 [==============================] - 1s 14ms/step - loss: 490.7164 - val_loss: 543.4828
Epoch 6/50
71/71 [==============================] - 1s 13ms/step - loss: 402.2884 - val_loss: 1128.7756
Epoch 7/50
71/71 [==============================] - 1s 13ms/step - loss: 379.3755 - val_loss: 508.2622
Epoch 8/50
71/71 [==============================] - 1s 14ms/step - loss: 332.5149 - val_loss: 1175.5696
Epoch 9/50
71/71 [==============================] - 1s 13ms/step - loss: 354.2022 - val_loss: 673.4705
Epoch 10/50
71/71 [==============================] - 1s 11ms/step - lo

##### Resumen del modelo

In [14]:
# Resumen del modelo para verificar la arquitectura
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 12, 64)            4352      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 12, 64)            8256      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 12, 32)            3104      
                                                                 
 flatten (Flatten)           (None, 384)               0         
                                                                 
 dense_27 (Dense)            (None, 32)                12320     
                                                                 
 dense_28 (Dense)            (None, 1)                 33        
                                                                 
Total params: 28065 (109.63 KB)
Trainable params: 2806